In [ ]:
import tensorflow as tf
import json
import numpy as np
import pandas as pd

from google.cloud import bigquery
from google.cloud import storage
from google.oauth2 import service_account

# from google.cloud import aiplatform


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix,classification_report
from google.cloud import aiplatform

# Reference
https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom-tabular-bq-managed-dataset.ipynb

In [ ]:
print(tf.__version__)

In [ ]:
#table_id='test_incident'
#table_id='validation_incident'
table_id='new_incident'
#table_id='train_incident'

ENDPOINT_ID="2469916532356939776"
MODEL_ID="2354195132556771328"

isLocalModel=True
isBQToCSV=True
localMapFile=True
gs_store_class="demo2-tf-incident-pongthorn"
gs_model_path="gs://demo2-tf-incident-pongthorn/demo_model_tf"

local_model_path='demo_model_tf'
# ! gsutil -m cp -r "gs://demo2-tf-incident-pongthorn/demo_model_tf" .


modelFile_endPoint=2
#1=local(dev)/gcs(clound function) and 2=endpoint on vertext

# projectId='smart-data-ml'
projectId='pongthorn'
dataset_id='SMartML'
REGION="asia-southeast1"

n_sample_run=0

# mycredentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
# bqclient= bigquery.Client(project=projectId,credentials=mycredentials)
bqclient= bigquery.Client(project=projectId)

if  table_id=='new_incident':
 unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']
 filePath='csv_dataset/Prediction_New_Incident.csv'
elif table_id=='test_incident' or table_id=='validation_incident':
 unUsedColtoPredict=['severity','id','severity_id','severity_name']  
 filePath=f'csv_dataset/Prediction_{table_id}.csv'


In [ ]:
if isBQToCSV:
    
    def download_table(bq_table_uri: str):

        prefix = "bq://"
        if bq_table_uri.startswith(prefix):
            bq_table_uri = bq_table_uri[len(prefix) :]

        table = bigquery.TableReference.from_string(bq_table_uri)
        rows = bqclient.list_rows(table)
        return rows.to_dataframe()

    dfNewData=download_table(f"{projectId}.{dataset_id}.{table_id}")
    dfNewData.to_csv(filePath,index=False)    
    
  
if n_sample_run>0:
 dfNewData=dfNewData.tail(n_sample_run)
print(dfNewData.info())


# Load and Map Data Test

In [ ]:

mapping_file="incident_severity_to_class.json"

def download_map_severity_class_as_dict():
 
    storage_client = storage.Client()
    buckdfNewData = storage_client.bucket(gs_store_class)
    blob = bucket.blob(mapping_file)
    blob.download_to_filename(mapping_file)
    with open(mapping_file, 'r') as file:
            return json.loads(file.read())

if  localMapFile==False:    
    map_sevirity_to_class= download_map_severity_class_as_dict()
else:
   with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)
                                  
                                  
print(map_sevirity_to_class)



In [ ]:
if  table_id=='new_incident':
   dfNewData['severity']= dfNewData['severity_name'].map(map_sevirity_to_class)
else:
    dfNewData=dfNewData.rename(columns={'label_multi_severity':'severity'})
    dfNewData=dfNewData.drop(columns=['label_binary_severity'])

if 'id' not in dfNewData.columns.to_list():
    dfNewData=dfNewData.reset_index(drop=True)
    dfNewData = dfNewData.reset_index(level=0)
    dfNewData.rename(columns={"index": "id"},inplace=True)
    dfNewData['id']=dfNewData['id']+1
print(f"Table: {table_id}")
print(filePath) 
print(dfNewData.info())

dfNewData.to_csv(f"{table_id}.csv",index=False)

In [ ]:
def plot_class(df):
    fig , ax = plt.subplots(figsize=(8,5))
    ax =sns.countplot(x='severity_name', data=df,)
    for p in ax.patches:
       ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))
    plt.title('Severity')
    plt.show()
    
plot_class(dfNewData)

# Classification Rerport

In [ ]:
def make_classification_report(dfResult):
    print(dfResult.tail())
    className=list(set().union(list(dfResult['severity'].unique()),list(dfResult['_predict_severity'].unique())))
    actualClass=[  f'actual-{x}' for x in  className]
    predictedlClass=[  f'predcited-{x}' for x in className]

    y_true=list(dfResult['severity'])
    y_pred=list(dfResult['_predict_severity'])
    cnf_matrix = confusion_matrix(y_true,y_pred)

    # #index=actual , column=prediction
    cm_df = pd.DataFrame(cnf_matrix,
                         index = actualClass, 
                         columns = predictedlClass)
    print(cm_df)

    print(classification_report(y_true, y_pred, labels=className))

# Load Model from Directory to Predict

In [ ]:
if  isLocalModel: # Window Enviroment
    PATH_FOLDER_ARTIFACTS=local_model_path   
else: # For running on Vertex AI Notebook t
    PATH_FOLDER_ARTIFACTS=gs_model_path
print(f"Load from {PATH_FOLDER_ARTIFACTS}")
model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)   
print(model.tensorflow_version)

# Prediction by model on entire rows at once

In [ ]:
if modelFile_endPoint==1 :
    pdPrediction=pd.DataFrame(columns=['_id','_predict_severity'])
    pdPrediction

    # print(model.summary())

    for  row_dict in dfNewData.to_dict(orient="records"):
          incident_id=row_dict['id']
          # print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
        
          for key_removed in unUsedColtoPredict:
           row_dict.pop(key_removed)
          print(row_dict)  

          input_tensor = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}
          predictionResult = model.predict(input_tensor)

          prob = tf.nn.softmax(predictionResult)
          prob_pct=(100 * prob)  
          _class = tf.argmax(predictionResult,-1).numpy()[0]
          
          pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':_class}])] )
          
          # print(input_tensor)    
          print(f"{prob_pct} %   as {_class}")     
          print("======================================================================================")

    dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
    dfPredictData=dfPredictData.drop(columns=['_id'])
    dfPredictData=dfPredictData[['id','_predict_severity','severity','severity_name']]
    print("Predicted all data completely")


# Registry and Deploy Model to VertextAI
- https://cloud.google.com/vertex-ai/docs/predictions/get-predictions#deploy_a_model_to_an_endpoint


## Load Model from Online EndPpint to Predict
- https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-online-predictions
- https://cloud.google.com/vertex-ai/docs/tutorials/tabular-bq-prediction/train-and-deploy-model
- https://cloud.google.com/vertex-ai/docs/tutorials/tabular-bq-prediction/make-prediction

# Load model from Vertext-Ai Model Registry

In [ ]:
#https://console.cloud.google.com/vertex-ai/endpoints?project=pongthorn
#https://console.cloud.google.com/vertex-ai/batch-predictions?project=pongthorn

MODEL_URI =  f'projects/{projectId}/locations/{REGION}/models/{MODEL_ID}'
print(MODEL_URI)

model_reg = aiplatform.Model(MODEL_URI)

model_name=model_reg.display_name
print(model_reg.uri," - ",model_name)

# Make a call to Endpoint

In [ ]:
isEndpointAvaiable=False

listRequestData=[]
try:
    
    endpoint = aiplatform.Endpoint(endpoint_name=f"projects/{projectId}/locations/{REGION}/endpoints/{ENDPOINT_ID}")
                                   
    # endpoint = aiplatform.Endpoint(endpoint_name=f"projects/{projectId}/locations/{REGION}/endpoints/{ENDPOINT_ID}"
    #                                ,credentials=mycredentials)
    print(endpoint)
    isEndpointAvaiable=True
except Exception as error:
    print(str(error))

# Create Instances and feed it to endpoint to predict

# Convert to tenfor format

In [ ]:
dfID=dfNewData[['id']]
dfID.reset_index(inplace=True,drop=True)
dfY=dfNewData[['severity','severity_name']]
dfY.reset_index(inplace=True,drop=True)
dfX=dfNewData.drop(columns=unUsedColtoPredict)
dfX.reset_index(inplace=True,drop=True)

In [ ]:
instances = []
for index,row in dfX.iterrows():
    dict_item=row.to_dict()
    instance = {}
    for key, value in dict_item.items():
        if key in unUsedColtoPredict:
            continue
        if value is None:
            value = ""
        instance[key] = [value]
        # instance[key] = value
    instances.append(instance)

print(len(instances))
print(instances[0])

# Prediction by endpoint on entire rows at once

In [ ]:
predictions = endpoint.predict(instances=instances)
predictedY = np.argmax(predictions.predictions, axis=1)

dfPredictedY=pd.DataFrame(data={'_predict_severity':predictedY})

In [ ]:
dfOnlinePred=pd.concat([dfID,dfPredictedY,dfY],axis=1)
dfOnlinePred.tail()

In [ ]:
make_classification_report(dfOnlinePred)

# Get explanations

In [ ]:
input_name = model.input_names
print("Model input name:", input_name)
output_name = model.output_names
print("Model output name:", output_name)

In [ ]:
# https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-online-predictions#interpret_explanation_results
# https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-online-predictions#example_output_for_predictions_and_explanations
# https://cloud.google.com/vertex-ai/docs/tabular-data/classification-explanations


listExplainModel=[]
def explain_model( instanceList):

    response = endpoint.explain(instances=instanceList, parameters={})

    for explanation in response.explanations:
        # print("Explanation")
        # Feature attributions.
        attributions = explanation.attributions
        item={}
        for attribution in attributions:
            item["baseline_output"]=attribution.baseline_output_value
            item["instance_output"]=attribution.instance_output_value
            item["approx_error"]=attribution.approximation_error
            item["baseline_output"]=attribution.baseline_output_value
            sum_feat=0;
            for name in input_name:
              feat_value= attribution.feature_attributions[name][0]
              sum_feat=sum_feat+feat_value
              # print(f"  {name} :{feat_value}")
              item[name]=feat_value
            # print(f"  The sum of all of the feature importance values(instance-baseline) = {sum_feat}")
            item["sum_feat_imp"]=sum_feat
        listExplainModel.append(item)   
        

explain_model(instances)

In [ ]:
dfExplainModelResult=pd.DataFrame(data=listExplainModel)
dfExplainModelResult

In [ ]:
dfABC=dfExplainModelResult[input_name].sum().to_frame().T
dfABC['dataset']=table_id
dfABC=dfABC.set_index('dataset')
dfABC

In [ ]:
# writer = pd.ExcelWriter(f'{table_id}_{model_name}_exp.xlsx')
# for i, dataframe in enumerate([dfExplainModelResult,dfABC]):
#     dataframe.to_excel(writer, sheet_name=f'Sheet{i + 1}')
# writer.save()

# Prediction individual row

In [ ]:
# if modelFile_endPoint==2 and isEndpointAvaiable==True :

#     for data in dfNewData.to_dict(orient="records"):
#         incident_id=data['id']
#         severity=data['severity']
#         severity_name=data['severity_name']
#         # convert dict to tf format
#         request_data = {key : [value] for key,value in data.items() }
#         for key_removed in unUsedColtoPredict:
#            request_data.pop(key_removed)
#         listRequestData.append(request_data)
#         print(request_data)

#         response = endpoint.predict([request_data])
#         y_predicted = np.argmax(response.predictions, axis=1)
#         predictionResult=response[0][0]
#         print(predictionResult)
#         print(y_predicted)

#         pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':y_predicted[0]}])] )
#         print("======================================================================================")

#     dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
#     dfPredictData=dfPredictData.drop(columns=['_id'])
#     dfPredictData=dfPredictData[['id','_predict_severity','severity','severity_name']]       
# else:
#      print("No endpoint to predict  as online predction")     